In [1]:
!git clone https://github.com/LC1332/luotuo-silk-road.git
%cd  luotuo-silk-road/TuoLing
!pip install -r requirements.txt 

Cloning into 'luotuo-silk-road'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 5), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (22/22), 25.97 MiB | 8.01 MiB/s, done.
/content/luotuo-silk-road/TuoLing
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-7nn7udvt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-7nn7udvt
  Resolved https://github.com/huggingface/peft.git to commit df71b84341ae1ab3bc9b0d5f906d7a524850b63b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/

In [15]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, device_map='auto')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [17]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "output/luotuoC.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/usr/local/lib/python3.9/dist-packages/peft/tuners/lora.py:175: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


# This model is a summarization model, you can simply copy your rich text into "input" in the next cell. The max input and output sequence length is 2048 currently. Please don't copy input or expect output text longer than this number for accuracy purpose. If you need to train or validate more domain specific data, please contact our team! Thank you!

# 本模型为summarization模型，您可以将对应长文本数据，直接复制到下面的输入中。当前模型输入及输出总长度为2048，请复制时不要将大于这个长度的文档放入，以免不准确。如果需要进一步验证数据或训练更多领域相关数据，请随时联系我们团队！谢谢！

In [50]:
from cover_alpaca2jsonl import format_example
def evaluate(instruction, input=None):
  with torch.no_grad():
    feature = format_example({'instruction': '请帮我总结以下内容:', 'output': '', 'input': f'{instruction}'})
    input_text = feature['context']
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    out = model.generate(
            input_ids=input_ids,
            max_length=2048,
            temperature=0
        )
    answer = tokenizer.decode(out[0])
    print(answer)

In [58]:
evaluate(input("您需要总结的长文本请直接复制在这里: "))

您需要总结的长文本请直接复制在这里: 美国硅谷银行 (Silicon Valley Bank, SVB ;矽谷银行)和“标志银行”(Signature Bank,又译“签名银行”)在几天内相继倒闭后,美国当局采取了紧急措施来支撑银行系统。这是美国自2008年金融海啸以来最大的银行倒闭事件,恐引起金融市场骨牌效应。硅谷银行因资不抵债在48小时内倒闭后,3月10日由美国联邦存款保险公司(FDIC)接管。美国财政部、联储局及联邦存款保险公司12日晚间发表联合声明指,正采取果断行动,以增强公众对银行体系的信心,并向硅谷银行存户保证,可以在13日周一取回所有存款。3月12日,美国财政部以存在系统性风险为由关闭总部设在纽约州的“标志银行”。该银行是加密货币公司的主要融资来源之一。美国总统拜登发表讲话,承诺将“竭尽所能”保护银行系统。但投资者担心其他银行可能仍会受到影响,引发全球股价大幅下跌。事件震荡全球金融市场,加拿大、英国等多个市场都受牵连。法兰克福、巴黎和米兰的股市大幅下跌。周一(3月13日)早些时候,西班牙桑坦德银行(Santander)和德国商业银行(Commerzbank)的股价一度下跌超过10%。极地资本(Polar Capital)基金经理乔治·戈德伯(George Godber)表示,市场下跌是因为“担心那里可能还有其他事情发生”。他说:“迫在眉睫的危机可能已经避免,但它提醒人们注意这样一个事实,即有一群公司的商业模式将在高利率环境中苦苦挣扎。”但他表示,事件对英国经济和英国市场的直接影响有限。BBC中文梳理这次危机的来龙去脉,以及对全球的影响。
美国硅谷银行 (Silicon Valley Bank, SVB ;矽谷银行)和“标志银行”(Signature Bank,又译“签名银行”)在几天内相继倒闭后,美国当局采取了紧急措施来支撑银行系统。这是美国自2008年金融海啸以来最大的银行倒闭事件,恐引起金融市场骨牌效应。硅谷银行因资不抵债在48小时内倒闭后,3月10日由美国联邦存款保险公司(FDIC)接管。美国财政部、联储局及联邦存款保险公司12日晚间发表联合声明指,正采取果断行动,以增强公众对银行体系的信心,并向硅谷银行存户保证,可以在13日周一取回所有存款。3月12日,美国财政部以存在系统性风险为由关闭总部设在纽约州的“标志银行”。该银行是加密货币公司的主要融

In [54]:
evaluate(input("Put your rich text here which you want to summarize it: "))

Put your rich text here which you want to summarize it: 第二次世界大战之后，美国是无可争辩的世界头号经济体，并且它相信自己在军事上也同样无可匹敌。然而在越南，美国尽管投入了大量的资金和人力，但是在经过至少八年的战斗之后还是被北越及其游击队同盟越南南方民族解放阵线（Viet Cong，惯称“越共”）所打败。3月29日是美国从越南最终撤军的50周年，我们访问了两位专家学者，了解美国到底是如何在越南战争中落败。 那是正值冷战时代的高峰，共产主义和资本主义大国正在全球展开对抗。 被二战拖垮至破产的法国曾试图保住他们在中南半岛（Indochina，另称“印度支那”）的殖民地未果，然后一场和平的会议中将如今的越南分割成北边的共产主义政权和南边由美国扶植的政府。 但是法国人的败退并未终结越南国内的冲突，而美国害怕一旦越南全国成为共产主义政权，那么它周边的国家也会一样。美国就是在这种恐惧的驱动下，卷入了一场持续十年并夺去数以百万计生命的战争。
第二次世界大战之后，美国是无可争辩的世界头号经济体，并且它相信自己在军事上也同样无可匹敌。然而在越南，美国尽管投入了大量的资金和人力，但是在经过至少八年的战斗之后还是被北越及其游击队同盟越南南方民族解放阵线（Viet Cong，惯称“越共”）所打败。3月29日是美国从越南最终撤军的50周年，我们访问了两位专家学者，了解美国到底是如何在越南战争中落败。 那是正值冷战时代的高峰，共产主义和资本主义大国正在全球展开对抗。 被二战拖垮至破产的法国曾试图保住他们在中南半岛（Indochina，另称“印度支那”）的殖民地未果，然后一场和平的会议中将如今的越南分割成北边的共产主义政权和南边由美国扶植的政府。 但是法国人的败退并未终结越南国内的冲突，而美国害怕一旦越南全国成为共产主义政权，那么它周边的国家也会一样。美国就是在这种恐惧的驱动下，卷入了一场持续十年并夺去数以百万计生命的战争。
Instruction: 请帮我总结以下内容:
Input: 第二次世界大战之后,美国是无可争辩的世界头号经济体,并且它相信自己在军事上也同样无可匹敌。然而在越南,美国尽管投入了大量的资金和人力,但是在经过至少八年的战斗之后还是被北越及其游击队同盟越南南方民族解放阵线(Viet Cong,惯称“越共”)所打败。3月29